In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tqdm.notebook import tqdm
import itertools

from adaptive_algos import SGD_TC
import helper_funcs as hf
import ObjectiveFunction as of

In [3]:
class PointDataset(Dataset):
    def __init__(self, seed, dim, size, bounds):
        np.random.seed(seed)
        self.data = [torch.tensor(np.random.rand(dim)*bounds - 0.5*bounds) for c in range(size)]
        self.data = torch.vstack(self.data)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]

def cyclic_loader(dataloader):
    while True:
        for data in dataloader:
            yield data

In [8]:
start = [1.0]
bounds = 10
pointdata = PointDataset(seed=1, dim=1, size=56, bounds=bounds)
batchsize = 4
dataloader = torch.utils.data.DataLoader(dataset=pointdata, batch_size=batchsize, shuffle=False)
train_cdl = cyclic_loader(dataloader)

opt_params={'lr': 0.05}
model_params={'start':start, 'bounds':bounds}
n_iterations = 100

model = of.Hodgkinson(**model_params)
opt = torch.optim.SGD(params=model.parameters(), **opt_params)
losses = [model(data=pointdata.data).item()]
params = [list(model.parameters())[0].detach().clone().numpy()]

for i in range(n_iterations):
    model.train()
    batch = next(train_cdl)
    opt.zero_grad()
    loss = model(data=batch)
    loss.backward()
    opt.step()

    full_loss = model(data=pointdata.data)
    losses.append(full_loss.item())
    param = list(model.parameters())[0].detach().clone()
    param = model.apply_period(param)
    params.append(param.numpy())

In [9]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(losses)), y=losses))
fig.update_layout(
    autosize=False,
    width=800,
    height=800
)
fig.show()

In [12]:
seed = 1 # determines function
n_runs = 1000
n_iterations = 100
dim, size, batchsize, bounds, = 1, 56, 12, 20
start = [1.0]
opt_params = {'lr': 0.05}
model_params = {'start':start, 'bounds':bounds}
dataset_params= {'seed': seed, 'dim':dim, 'size':size, 'bounds':bounds}
final_params = []

pointdata = PointDataset(**dataset_params)
for i in tqdm(range(n_runs)):

    # Shuffle false gives all the same results as expected
    torch.manual_seed(seed+i)
    dataloader = torch.utils.data.DataLoader(dataset=pointdata, batch_size=batchsize, shuffle=True)
    train_cdl = cyclic_loader(dataloader)
    model = of.Hodgkinson(**model_params)
    opt = torch.optim.SGD(params=model.parameters(), **opt_params)

    for j in range(n_iterations):
        batch = next(train_cdl)
        opt.zero_grad()
        loss = model(data=batch)
        loss.backward()
        opt.step()

    final_param = list(model.parameters())[0].detach().clone()
    final_param = model.apply_period(final_param).numpy()  
    final_params.append(final_param)

xlim, res = (-bounds/2 - 1, bounds/2 + 1), 0.01
X = np.arange(xlim[0], xlim[1], res)
Z = [float(model.forward(data=pointdata.data, X=torch.Tensor([_]))) for _ in X]

fig1 = make_subplots(specs=[[{"secondary_y": True}]])
fig1.add_trace(
    go.Scatter(x=X, y=Z, line=dict(color='black')),
    secondary_y=False
)
hist_fig = px.histogram(final_params, opacity=0.2, 
                        nbins=120, histnorm='probability density')
fig1.add_trace(
    hist_fig.data[0],
    secondary_y=True
)
fig1.update_layout(
    title= 'SGD with Function Seed'.format(seed),
    width=800,
    height=800
)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [13]:
# SGD_TC
from adaptive_algos import SGD_TC
seed = 1 # determines function
n_runs = 1000
max_iterations = 100
dim, size, batchsize, bounds, = 1, 56, 12, 20
start = [1.0]
annealer = lambda progress: 1
opt_params={'lr': 0.05, 'height': 1.0, 'width': bounds/20,
            'annealer': annealer, 'n_epochs': max_iterations}
model_params = {'start':start, 'bounds':bounds}
dataset_params= {'seed': seed, 'dim':dim, 'size':size, 'bounds':bounds}
final_params = []

pointdata = PointDataset(**dataset_params)
for i in tqdm(range(n_runs)):

    # Shuffle false gives all the same results as expected
    torch.manual_seed(seed+i)
    dataloader = torch.utils.data.DataLoader(dataset=pointdata, batch_size=batchsize, shuffle=True)
    train_cdl = cyclic_loader(dataloader)
    model = of.Hodgkinson(**model_params)
    opt = SGD_TC(params=model.parameters(), func=model, **opt_params)

    for j in range(max_iterations):
        batch = next(train_cdl)
        opt.zero_grad()
        loss = model(data=batch)
        loss.backward()
        opt.step()

    final_param = list(model.parameters())[0].detach().clone()
    final_param = model.apply_period(final_param).numpy()  
    final_params.append(final_param)

xlim, res = (-bounds/2 - 1, bounds/2 + 1), 0.01
X = np.arange(xlim[0], xlim[1], res)
Z = [float(model.forward(data=pointdata.data, X=torch.Tensor([_]))) for _ in X]

fig2 = make_subplots(specs=[[{"secondary_y": True}]])
fig2.add_trace(
    go.Scatter(x=X, y=Z, line=dict(color='black')),
    secondary_y=False
)
hist_fig = px.histogram(final_params, opacity=0.2, 
                        nbins=120, histnorm='probability density')
fig2.add_trace(
    hist_fig.data[0],
    secondary_y=True
)
fig2.update_layout(
    title= 'SGD_TC with Function Seed'.format(seed),
    width=800,
    height=800
)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [14]:
# SGD_TC with annealing
seed = 1 # determines function
n_runs = 1000
max_iterations = 100
dim, size, batchsize, bounds, = 1, 56, 12, 20
start = [1.0]
annealer = lambda progress: 1 - progress
opt_params={'lr': 0.05, 'height': 1.0, 'width': bounds/20,
            'annealer': annealer, 'n_epochs': max_iterations}
model_params = {'start':start, 'bounds':bounds}
dataset_params= {'seed': seed, 'dim':dim, 'size':size, 'bounds':bounds}
final_params = []

pointdata = PointDataset(**dataset_params)
for i in tqdm(range(n_runs)):

    # Shuffle false gives all the same results as expected
    torch.manual_seed(seed+i)
    dataloader = torch.utils.data.DataLoader(dataset=pointdata, batch_size=batchsize, shuffle=True)
    train_cdl = cyclic_loader(dataloader)
    model = of.Hodgkinson(**model_params)
    opt = SGD_TC(params=model.parameters(), func=model, **opt_params)

    for j in range(max_iterations):
        batch = next(train_cdl)
        opt.zero_grad()
        loss = model(data=batch)
        loss.backward()
        opt.step()

    final_param = list(model.parameters())[0].detach().clone()
    final_param = model.apply_period(final_param).numpy()  
    final_params.append(final_param)

xlim, res = (-bounds/2 - 1, bounds/2 + 1), 0.01
X = np.arange(xlim[0], xlim[1], res)
Z = [float(model.forward(data=pointdata.data, X=torch.Tensor([_]))) for _ in X]

fig3 = make_subplots(specs=[[{"secondary_y": True}]])
fig3.add_trace(
    go.Scatter(x=X, y=Z, line=dict(color='black')),
    secondary_y=False
)
hist_fig = px.histogram(final_params, opacity=0.2, 
                        nbins=120, histnorm='probability density')
fig3.add_trace(
    hist_fig.data[0],
    secondary_y=True
)
fig3.update_layout(
    title= 'SGD_TC Annealed with Function Seed {}'.format(seed),
    width=800,
    height=800
)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [9]:
hf.figures_to_html([fig1, fig2, fig3], 
                    'proof_of_concepts_batch12.html')


In [ ]:
# Escape time analysis
"""
Starting in a well of depth d and FWHM w, 
how does the first escape time vary with d and w?

Do this with artificial noise first.
"""






In [ ]:
# Fixed alpha - demonstrate exploration
# Bigger alpha - converge

"""
- proof of concept 1d and 2d landscapes, statistics of convergence of minima
        - different batch size?
        - different lr?
- linear regression
- different alphas

- comparison with metadynamics/tempering, real problems

"""


